<a href="https://colab.research.google.com/github/andrewmlu/super-stable-rep/blob/main/CS242_Project_Scratchpad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir("/content/drive/My Drive/!!!Drive/CS242-SD-imgs")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [89]:
import requests
import json
from PIL import Image
import time
import re
import math
import numpy as np
import random
import datetime
import csv

In [92]:
NUM_IMAGES_PER_PROMPT = 24

In [64]:
url = "https://stablediffusionapi.com/api/v3/text2img"

def call_api(prompt, negative_prompt = None):

  # temp api key included
  payload = json.dumps({
    "key": "tgc1aN0Kk9ieQBgoVT1dssH0hvpeEap2YOMBEJIHYyUo2wnenimTpln5tnPO",
    "prompt": prompt,
    "negative_prompt": negative_prompt,
    "width": "512",
    "height": "512",
    "samples": "4",
    "num_inference_steps": "31",
    "seed": None,
    "guidance_scale": 7.5,
    "safety_checker": "yes",
    "multi_lingual": "no",
    "panorama": "no",
    "self_attention": "no",
    "upscale": "no",
    "embeddings_model": None,
    "webhook": None,
    "track_id": None
  })

  headers = {
    'Content-Type': 'application/json'
  }

  response = requests.request("POST", url, headers=headers, data=payload)

  return response


In [65]:
# response = call_api("golden labrador in a field of sunflowers")
# img_srcs = json.loads(response.text)['output']
# img_srcs

In [76]:
# GPT-4 turbo
def save_img(url, filepath=''):

  # This statement requests the resource at
  # the given link, extracts its contents
  # and saves it in a variable
  data = requests.get(url).content
  # print(data)

  # Opening a new file named img with extension .png
  # This file would store the data of the image file
  filename = filepath + re.search(r'[^/]*$', url).group()
  f = open(filename,'wb')

  # Storing the image data inside the data variable to the file
  f.write(data)
  f.close()

  # Opening the saved image and displaying it
  # img = Image.open(filename)
  # img.show()


In [106]:
def save_imgs_from_prompt(prompt, negative_prompt=None, num_imgs=NUM_IMAGES_PER_PROMPT, filepath=''):
  count = 0
  while count < NUM_IMAGES_PER_PROMPT: # cannot simply use for-loop, instead use
    print(count)
    response = call_api(prompt, negative_prompt) # minor debug: put response, img_srcs inside loop
    print(response.text)
    img_srcs = json.loads(response.text)['output']
    if json.loads(response.text)['status'] == "success":
      for url in img_srcs:
        print(url)
        save_img(url, filepath=filepath)
        save_img_caption(prompt, url, filepath=filepath)
      count += len(img_srcs)

In [107]:
!ls

batch1	CS242-SD-imgs  img-captions.csv  img-captions.txt  synsets.txt


In [108]:
save_imgs_from_prompt("golden labrador in a field of sunflowers", filepath='batch1/')

0
{"status":"processing","tip":"Get 20x faster image generation using enterprise plan. Click here : https:\/\/stablediffusionapi.com\/enterprise","tip_1":"Your image is processing in background, you can get this image using fetch API: here is link for fetch api : https:\/\/stablediffusionapi.com\/docs\/community-models-api-v4\/dreamboothfetchqueimg","eta":38.69,"messege":"Try to fetch request after seconds estimated","fetch_result":"https:\/\/stablediffusionapi.com\/api\/v3\/fetch\/56320095","id":56320095,"output":[],"meta":{"H":512,"W":512,"enable_attention_slicing":"true","file_prefix":"56ef8bad-7a27-4e3e-91c4-5a97e603e0c1","guidance_scale":7.5,"instant_response":"no","model":"runwayml\/stable-diffusion-v1-5","n_samples":4,"negative_prompt":"low quality","outdir":"out","prompt":"golden labrador in a field of sunflowers","revision":"fp16","safetychecker":"yes","seed":1542448286,"steps":30,"vae":"stabilityai\/sd-vae-ft-mse"}}
0
{"status":"processing","tip":"Get 20x faster image generat

In [ ]:
from transformers import CLIPProcessor, CLIPModel

# GPT-4 turbo
# Load the pre-trained CLIP model
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Text string
text = "a banana sitting at the beach"

# Prepare the text
inputs = processor(text=text, return_tensors="pt")

# Perform the encoding
text_features = model.get_text_features(**inputs)

# The text_features tensor is the encoded representation of your text
print(text_features)

In [ ]:
# Text string
text = ["a banana sitting at the beach", "two bananas sitting at the beach", "a dog sitting at the becah"]

# Prepare the text
inputs = processor(text=text, return_tensors="pt", padding=True)

# Perform the encoding
text_features = model.get_text_features(**inputs)

# The text_features tensor is the encoded representation of your text
print(text_features)

tensor([[ 0.1699,  0.3451, -0.0828,  ...,  0.2795, -0.0243, -0.0554],
        [ 0.2219,  0.2785, -0.1488,  ...,  0.2776, -0.0453, -0.1471],
        [-0.0926, -0.1647, -0.3049,  ..., -0.2542, -0.1363,  0.1552]],
       grad_fn=<MmBackward0>)


In [ ]:
text_features[0] @ text_features[1]

tensor(56.8696, grad_fn=<DotBackward0>)

In [ ]:
text_features[0] @ text_features[2]

tensor(49.9023, grad_fn=<DotBackward0>)

In [ ]:
text_features[1] @ text_features[2]

tensor(49.9115, grad_fn=<DotBackward0>)

In [37]:
# GPT-4 turbo (mostly)
from sklearn.metrics.pairwise import cosine_similarity

# Assuming text_features is a tensor with encoded text features
# Convert to numpy array if it's a PyTorch tensor
text_features_np = text_features.detach().numpy()

# Calculate cosine similarity
# This will return a matrix of similarities between each pair of texts
similarity_matrix = cosine_similarity(text_features_np)
np.fill_diagonal(similarity_matrix, np.NINF)

print(similarity_matrix[0])
print(np.argmax(similarity_matrix[0]))

KeyboardInterrupt: ignored

In [38]:
# strings = [f"a {1} next to a tree",
#            f"a {1} on the left of a building",
#            f"a {1} on the right of a wall",
#            f"a {1} in a bowl",
#            f"a {1} on top of a hill",
#            f"a {1} held in a hand",
#            f"a picnic table styled with {1}",
#            f"a close up of a {1}",
#            f"a crowd looking upon a {1}",
#            f"a nice picture of two {1}"]

# GPT-4 turbo
# Global list of string templates
strings = [
    "a {} next to a tree",
    "a {} on the left of a building",
    "a {} on the right of a wall",
    "a {} in a bowl",
    "a {} on top of a hill",
    "a {} held in a hand",
    "a picnic table styled with {}",
    "a close up of a {}",
    "a crowd looking upon a {}",
    "a nice picture of two {}"
]

def select_prompt(index, variable):
    # Use the global strings variable
    if 0 <= index < len(strings):
        return strings[index].format(variable)
    else:
        return "Invalid index"

# Example usage
result = select_prompt(2, "apple")
print(result)


a apple on the right of a wall


In [28]:
synsets = []
with open('synsets.txt', 'r') as f:
  # extract synsets, and create list of lists
  synsets = [re.search(r":\s(.*)", synset).group(1).split(sep=", ") for synset in f]  # https://stackoverflow.com/questions/10939950/simple-python-regex-match-after-colon

In [ ]:

print(synsets)
for synset in synsets:
  for i in range(NUM_IMAGES_PER_PROMPT):
    prompt = random.choice(synset)
    print(f"a {prompt} next to a tree")

In [84]:
def save_img_caption(prompt, url, filepath='', filetype='.png'):
  timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
  src = filepath + re.search(r'[^/]*$', url).group()
  # using open('w') would truncate/delete existing data, using 'a' for append instead
  with open('img-captions.csv', 'a', newline='') as f: # https://stackoverflow.com/questions/6159900/correct-way-to-write-line-to-file

    w = csv.writer(f) # GPT-4 turbo for csv
    w.writerow([timestamp, src, prompt])
      # f.write(re.sub(filetype + '$', '', src) + ": " + prompt) # GPT-4 turbo, if needed to remove filetype

